# Holoview prototype code
Goals here are to learn how holoviews work and write something that is functionally rendering the number of points we want to in the manner that looks feasible given the documentation, and to attempt 0th version of other drill-down features.
Focus is on speed of implementation, and getting contact with performance space we are working with using the Holoviews/Datashader/Bokeh stack in a notebook.
Not goals: Connect to actual data sources (vector DB, infer/predict output files, etc)

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import KDTree, Delaunay
import holoviews as hv
from holoviews.operation.datashader import rasterize
from holoviews.streams import RangeXY
from datetime import datetime

hv.extension("bokeh")

# Input Data
Generate a random gaussian splat with some Index numbers so we have something to work with

In [ ]:
n = 10_000_000
# n = 1_000_000
# n = 100_000

d = 2
name_v = ["x", "y", "z"][:d]
mean_v = [0.0, 0.0, 0.0][:d]
stdev_v = [1.0, 1.0, 1.0][:d]
params = zip(name_v, mean_v, stdev_v)

index = np.arange(n)
columns = {axis_name: np.random.normal(loc=mean, scale=stdev, size=n) for axis_name, mean, stdev in params}
df = pd.DataFrame(columns, index=index)
# df.iloc[0:3]

# A Quad tree
If you want to try the slowest version run this cell to define the datastructure

In [ ]:
# A quad-tree we can hack on
import numpy as np
import cython
from cython.cimports.libc.math import sqrt

# Point
Point = cython.struct(x=cython.float, y=cython.float, index=cython.int)


@cython.cfunc
def distance(p1: Point, p2: Point) -> float:
    x = p1.x - p2.x
    y = p1.y - p2.y
    return sqrt(x * x + y * y)


# Rect
Rect = cython.struct(min=Point, max=Point)


@cython.inline
@cython.cfunc
def contains(r: Rect, p: Point) -> bool:
    return p.x >= r.min.x and p.x < r.max.x and p.y >= r.min.y and p.y < r.max.y


@cython.inline
@cython.cfunc
def intersects(r1: Rect, r2: Rect) -> bool:
    return r1.min.x < r2.max.x or r2.min.x < r1.max.x or r1.min.y > r2.max.y or r2.min.y < r1.max.y


# True if R1 contains r2 entirely
@cython.inline
@cython.cfunc
def rect_contains(r1: Rect, r2: Rect) -> bool:
    return r1.min.x < r2.min.x and r1.max.x > r2.max.x and r1.min.y < r2.min.y and r1.max.y > r2.max.y


@cython.inline
@cython.cfunc
def center(r: Rect) -> Point:
    return Point((r.min.x + r.max.x) / 2.0, (r.min.y + r.max.y) / 2.0)


# Tree
max_points = 10000
Treenode = cython.struct(
    boundary=Rect,
    point_count=cython.int,
    points_idx=cython.int,
    points=Point[max_points],
    center=Point,
    se=cython.pointer,
    nw=cython.pointer,
    ne=cython.pointer,
    sw=cython.pointer,
)


@cython.cfunc
def make_treenode(rect: Rect) -> Treenode:
    node = Treenode(rect, 0, 0)
    node.center = center(rect)
    return node


@cython.cfunc
def divide(tree: Treenode):
    n_midpoint = Point(tree.center.x, tree.boundary.max.y)
    s_midpoint = Point(tree.center.x, tree.boundary.min.y)
    e_midpoint = Point(tree.boundary.max.x, tree.center.y)
    w_midpoint = Point(tree.boundary.min.x, tree.center.y)

    tree.sw = make_treenode(Rect(min=tree.boundary.min, max=tree.center))
    tree.ne = make_treenode(Rect(min=tree.center, max=tree.boundary.max))
    tree.se = make_treenode(Rect(min=s_midpoint, max=e_midpoint))
    tree.nw = make_treenode(Rect(min=w_midpoint, max=n_midpoint))


@cython.cfunc
def _insert(tree: Treenode, point: Point):
    # We're definitely inserting here or or children
    # Each nodes point_count is how many points are
    # at or below that level
    tree.point_count += 1

    # insert into us if there is room
    if tree.points is None:
        tree.points = [Point(0, 0)] * max_points
        tree.points_idx = 0

    if tree.points_idx < max_points:
        tree.points[tree.points_idx] = point
        tree.points_idx += 1
        return True

    # Divide if we don't have child trees yet
    if tree.nw is None:
        divide(tree)

    if point.x < tree.center.x:
        if point.y < tree.center.y:
            return _insert(tree.sw, point)
        else:
            return _insert(tree.nw, point)
    else:
        if point.y < tree.center.y:
            return _insert(tree.se, point)
        else:
            return _insert(tree.ne, point)


@cython.inline
@cython.cfunc
def insert(tree: Treenode, point: Point):
    # Reject points outside the tree
    if not contains(tree.boundary, point):
        return False
    # Assign every point an index, then delegate all work to helper
    point.index = tree.point_count

    return _insert(tree, point)


@cython.cfunc
def insert_all(tree: Treenode, points_x: np.array, points_y: np.array):
    length = len(points_x)
    for i in range(length):
        _insert(tree, Point(points_x[i], points_y[i], i))


@cython.inline
@cython.cfunc
def box_query(tree: Treenode, box: Rect, points: list[int]) -> bool:
    if not intersects(tree.boundary, box):
        return False
    return _box_query(tree, box, points)


@cython.inline
@cython.cfunc
def _all_query(tree: Treenode, box: Rect, points: list[int]):
    if tree.points_idx != 0:
        for index in range(tree.points_idx):
            points.append(tree.points[index].index)

    if tree.nw is not None:
        _all_query(tree.sw, box, points)
        _all_query(tree.ne, box, points)
        _all_query(tree.se, box, points)
        _all_query(tree.nw, box, points)


@cython.cfunc
def _box_query(tree: Treenode, box: Rect, points: list[int]) -> bool:
    # If this node is entirely inside the search box
    # just get everything
    if rect_contains(box, tree.boundary):
        _all_query(tree, box, points)
        return

    if tree.points_idx != 0:
        for index in range(tree.points_idx):
            if contains(box, tree.points[index]):
                points.append(tree.points[index].index)

    if tree.nw is not None:
        box_query(tree.sw, box, points)
        box_query(tree.ne, box, points)
        box_query(tree.se, box, points)
        box_query(tree.nw, box, points)

    return True


def printpt(p: Point):
    print(f"({p.x},{p.y})")


@cython.cfunc
def create(df: pd.DataFrame) -> Treenode:
    x = df["x"].to_numpy()
    y = df["y"].to_numpy()

    root = make_treenode(Rect(Point(np.min(x), np.min(y)), Point(np.max(x), np.max(y))))

    insert_all(root, x, y)
    return root


def profileme():
    create(df)

In [ ]:
a = make_treenode(Rect(Point(0, 0), Point(1, 1)))
insert(a, Point(0.5, 0.5))
insert(a, Point(0.1, 0.5))
insert(a, Point(0.5, 0.1))
insert(a, Point(0.1, 0.1))
insert(a, Point(0.1, 0.2))
insert(a, Point(0.1, 0.3))
b = []
box_query(a, Rect(Point(-1, -1), Point(2, 2)), b)
print(b)

# Choose a tree implementation
Only run one cell here. Whatever you pick is used for CPU culling of data sent to datashader

In [ ]:
# Quad tree (slowest, uncomment)
# tree = create(df)

In [ ]:
# no tree
tree = df

In [ ]:
# kdtree (fastest)
# tree = KDTree(df)

# Make a DynamicMap
This will select a subset of our points based on a box (x1, y1, x2, y2).

In [ ]:
if isinstance(tree, Treenode):

    def box_select_indexes(tree: Treenode, x_range, y_range):
        points = []
        box_query(tree, Rect(Point(x_range[0], y_range[0]), Point(x_range[1], y_range[1])), points)
        points = np.array(points)
        return points

elif isinstance(tree, KDTree):

    def box_select_indexes(tree: KDTree, x_range, y_range):
        # Find center
        xc = (x_range[0] + x_range[1]) / 2.0
        yc = (y_range[0] + y_range[1]) / 2.0
        query_pt = [xc, yc]

        # Find larger of  half-width and half-height to use as our search radius.
        radius = np.max([np.max(x_range) - xc, np.max(y_range) - yc])

        try:
            return tree.query_ball_point(query_pt, radius, p=np.inf)
        except:
            return np.array([])
else:

    def box_select_indexes(tree, x_range, y_range):
        return np.array(range(len(tree)))


def select_values(x_range, y_range):
    if x_range is None or y_range is None:
        return hv.Points([])

    if isinstance(tree, KDTree) or isinstance(tree, Treenode):
        start = datetime.now()
        point_indexes = box_select_indexes(tree, x_range, y_range)
        lookup_time = datetime.now() - start

    start = datetime.now()

    if isinstance(tree, KDTree) or isinstance(tree, Treenode):
        points = hv.Points(df.iloc[point_indexes])
    else:
        points = hv.Points(df.iloc[:])

    convert_time = datetime.now() - start

    print(f"Points Rendered: {len(points):,.0f}", end=", ")

    convert_ms = convert_time.total_seconds() * 1000
    if isinstance(tree, KDTree) or isinstance(tree, Treenode):
        lookup_ms = lookup_time.total_seconds() * 1000
        print(f"lookup_time: {lookup_ms:.0f} ms", end=", ")
        print(f"convert_time: {convert_ms:.0f} ms", end=", ")
        print(f"total_time: {lookup_ms + convert_ms:.0f} ms")
    else:
        print(f"convert_time: {convert_ms:.0f} ms")

    return points


def select_cb(index):
    if not index:
        return
    print(index)
    return "Got here"


def tap_cb(x, y):
    dist, id = tree.query([x, y])
    print(dist, id)

    return f"Nearest Point ID: {id}"


def square_cb(bounds, x_selection, y_selection):
    print(f"Rect with bounds: {bounds}, x_selection:{x_selection}, y_selection: {y_selection}")

    point_indexes = box_select_indexes(tree, x_selection, y_selection)
    points = df.iloc[point_indexes]
    in_points = points.loc[
        (points["x"] >= x_selection[0])
        & (points["x"] <= x_selection[1])
        & (points["y"] >= y_selection[0])
        & (points["y"] <= y_selection[1])
    ]

    return f"{len(in_points):,.0f} points"


def lasso_cb(geometry):
    #   - Search the KD tree for the maximal bounding square of the poly handed in
    boundary = pd.DataFrame(geometry, columns=["x", "y"])
    x_range = (boundary["x"].min(), boundary["x"].max())
    y_range = (boundary["y"].min(), boundary["y"].max())

    point_indexes = box_select_indexes(tree, x_range, y_range)

    #   - Convert indexes -> points
    points = df.iloc[point_indexes]

    #   - Delauny triangulate the poly
    tri = Delaunay(boundary)

    #   - Use find_simplex to filter points out
    in_points = points.loc[(tri.find_simplex(points) != -1)]

    # Do something "cool"
    return f"{len(in_points):,.0f} points"


# V2: Look into listening on one of these streams, pumping summary into a plotting pane that updates.

hv_options = {
    "tools": ["box_select", "lasso_select", "tap"],
    "width": 500,
    "height": 500,
}

rangexy = RangeXY()
dm = hv.DynamicMap(select_values, streams=[rangexy])
# hv.streams.BoundsXY(source=dm, popup="Used Box Select")
hv.streams.Lasso(source=dm, popup=lasso_cb)
hv.streams.Tap(source=dm, popup=tap_cb)
# hv.streams.Selection1D(source=dm, popup=select_cb)
# hv.streams.SelectionExpr(source=dm, popup="SelectionExpr")
hv.streams.SelectionXY(source=dm, popup=square_cb)

rasterize(dm).opts(**hv_options)
# dm.opts(**hv_options)
# dm.opts(width=500, height=500)